In [ ]:
import time
from IPython.display import clear_output, display

import rospy
from sensor_msgs.msg import Imu

import numpy as np
import torch

In [ ]:
gyro_fps=400
accel_fps=250

In [ ]:
def skew2vec(W):
    assert W.shape==(3,3)
    w=torch.tensor([(W[2,1]-W[1,2])/2,
                  (W[0,2]-W[2,0])/2,
                  (W[1,0]-W[0,1])/2,
                   ])
    return w

def vec2skew(w):
    assert w.shape==(3,)
    W=torch.tensor(
                  [[0,-1*w[2],w[1]],
                  [w[2],0,-1*w[0]],
                  [-1*w[1],w[0],0]
                  ])
    return W

In [ ]:
def r2w(r):
    # exponential coordinate to w,theta coordinate
    theta = torch.linalg.norm(r)
    assert 0<theta and theta<np.pi
    w = r/theta
    
    if abs(theta-0)<0.1 or abs(theta-np.pi)<0.1:
        print("warning!! theta near singularity")
    
    return theta , w

def w2r(theta,w):
    assert 0<theta and theta<np.pi
    return theta*w

In [ ]:
def Rodrigues(theta,W):
    return torch.eye(3)+(torch.sin(theta)*W)+((1-torch.cos(theta))*(W@W))

def matlog(R):
    tr = torch.trace(R)
    assert abs(torch.linalg.det(R)-1)<0.0001
    assert tr<3 and tr>-1
    if abs(tr-3)<0.1 or abs(tr+1)<0.1:
        print("warning!! R near singularity")
    
    theta=torch.arccos((tr-1)/2)
    W=(R-R.T)/(2*torch.sin(theta))
    
    return theta,W

In [ ]:
def aJacobian(theta,W):
    rmat=theta*W
    A=torch.eye(3) - (((1-torch.cos(theta))/(theta**2))*rmat) + (((theta-torch.sin(theta))/(theta**3))*(rmat@rmat))         
    return A

In [ ]:
def dynamic_model(r,wb,dt):
    assert type(r)==torch.Tensor
    wb_=torch.tensor(wb,dtype=float)
    theta,w=r2w(r)
    W=vec2skew(w)
    A=aJacobian(theta,W)
    Ainv=torch.inverse(A)
    return r+(dt*Ainv@wb_)

In [ ]:
def measurement_model(r):
    g=torch.tensor([0,0,-9.8],dtype=float)
    
    theta,w=r2w(r)
    W=vec2skew(w)
    R=Rodrigues(theta,W)
    Rinv=torch.inverse(R)
    zhat=Rinv@g
    return zhat


In [ ]:
def tensor_jacobian(T,r):
    shape=T.shape
    T_=T.view(-1)
    T_length=T_.shape[0]
    r_length=r.shape[0]
    J=torch.zeros((T_length,r_length),dtype=float)
    
    for i in range(T_length):
        if not r.grad==None:
            r.grad.zero_()
        T_[i].backward(retain_graph=True)
        J[i,:]=r.grad
    
    r.grad.zero_()
    
    J=J.view(*shape,r_length)
    return J
        
        
        
    
    
    
    

In [ ]:
def EKF(r: torch.Tensor,cov: torch.Tensor, 
        wb: np.ndarray, a: np.ndarray, dt) -> torch.Tensor:
    S=0.02*torch.eye(3,dtype=float) #dynamics noise
    Q=30*torch.eye(3,dtype=float) #measurement noise
    
    g=dynamic_model(r,wb,dt)
    G=tensor_jacobian(g,r)
    r_bar=g.clone().detach().requires_grad_(True)
    h=measurement_model(r_bar)
    H=tensor_jacobian(h,r_bar)
    
    with torch.no_grad():
        cov_bar = G@cov@G.T + S
        K = cov_bar@H.T@torch.inverse(H@cov_bar@H.T+Q)
        r_next = r_bar + (K@(torch.tensor(a,dtype=float)-h))
        cov_next = (torch.eye(3,dtype=float)-(K@H))@cov_bar
    
    #print(a,h)
    
    
    return r_next,cov_next

In [ ]:
class ReaderConfig:
    def __init__(self,
                 r0=np.array([np.pi/2,0,0]),
                 cov0=1.*np.eye(3),
                 accel_scale=np.array([1.017125065,1.02456874,1.018181818]),
                 accel_bias=np.array([0.01525687597,-0.445687402,-0.3309090909]),
                 dt=1/200,
                ):
        #accelerometer calibration ax+b
        self.accel_scale=accel_scale
        self.accel_bias=accel_bias
        self.r0=torch.tensor(r0,dtype=float)
        self.cov0=torch.tensor(cov0,dtype=float)
        self.dt=dt
        

In [ ]:
class IMU_reader:
    def __init__(self,config):
        #accelerometer calibration ax+b
        self.config=config
        self.a=self.config.accel_scale
        self.b=self.config.accel_bias
        
        self.accel = np.zeros(3)
        self.gyro = np.zeros(3)
        rospy.init_node('listener', anonymous=True)
        rospy.Subscriber("/camera/accel/sample", Imu, self.accel_callback)
        rospy.Subscriber("/camera/gyro/sample", Imu, self.gyro_callback)
        
        self.r=self.config.r0.clone().requires_grad_(True)
        self.cov=self.config.cov0.clone().requires_grad_(True)
        

    def accel_callback(self,data):
        v=data.linear_acceleration
        v=np.array([v.x,v.y,v.z])
        self.accel=self.a*v+self.b

    def gyro_callback(self,data):
        t0=time.time()
        v=data.angular_velocity
        v=np.array([v.x,v.y,v.z])
        self.gyro=v

        r_,cov_=EKF(self.r,self.cov,self.gyro,self.accel,self.config.dt)
        self.r=r_.clone().detach().requires_grad_(True)
        self.cov=cov_.clone().detach().requires_grad_(True)
        t1=time.time()
        
        clear_output(wait=True)
        display(f"({self.r[0].item():.3f},{self.r[1].item():.3f},{self.r[2].item():.3f}) std: {torch.linalg.det(self.cov):.3f}, time={t1-t0:.4f}")          
        #print(self.accel)

In [ ]:
config=ReaderConfig()
imur=IMU_reader(config)